In [1]:
import torch
from torch_geometric.nn import GCNConv
from torch.nn import Linear
from torch_geometric.nn import global_mean_pool
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from pool_class import *


In [11]:
dataset = TUDataset('data', name='MUTAG')
dataset = dataset.shuffle()
n = (len(dataset) + 9) // 10
test_dataset = dataset[:n]
val_dataset = dataset[n:2 * n]
train_dataset = dataset[2 * n:]
data = train_dataset[0]
data

Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])

In [12]:
# Test whether the message can flow (gradient can be updated automatically)
class dispooling_GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.pool1 = pooling(hidden_channels, score_method=2, p1=0.2, p2=0.95, aggregate_score_method='avg')
        # self.poo1 = EdgePooling(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index).relu()
        edge_index, _, h, _, _ = self.pool1(edge_index, h)
        # h, edge_index, _, _ = self.pool1(x, edge_index, batch=None)
        h = self.conv2(h, edge_index).relu()
        h = self.conv3(h, edge_index).relu()
        h = self.lin(h).relu()
        h = global_mean_pool(h, batch=None)
        return F.log_softmax(h, dim=-1)

In [22]:
model = dispooling_GCN(in_channels=dataset.num_node_features, hidden_channels=20, out_channels=dataset.num_classes)
model.forward(data.x, data.edge_index)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    for data in train_dataset:
        optimizer.zero_grad() 
        out = model(data.x, data.edge_index)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
    return(loss)


def test():
    model.eval()
    correct = 0
    for data in test_dataset:
        pred = model(data.x, data.edge_index).argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct / len(test_dataset)


def val():
    model.eval()
    correct = 0
    for data in val_dataset:
        pred = model(data.x, data.edge_index).argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct / len(test_dataset)



best_val_acc = 0
for epoch in range(1, 171):
    train_loss = train()
    val_acc = val()
    if val_acc > best_val_acc:
        test_acc = test()
        best_val_acc = val_acc
    if epoch % 10 == 1:
        print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, '
            f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Loss: 0.3918, Val Acc: 0.5789, Test Acc: 0.3684
Epoch: 011, Train Loss: 0.6931, Val Acc: 0.7368, Test Acc: 0.7368
Epoch: 021, Train Loss: 0.6931, Val Acc: 0.7895, Test Acc: 0.7368
Epoch: 031, Train Loss: 0.6931, Val Acc: 0.6842, Test Acc: 0.7368
Epoch: 041, Train Loss: 0.6931, Val Acc: 0.8421, Test Acc: 0.7895
Epoch: 051, Train Loss: 0.6931, Val Acc: 0.7895, Test Acc: 0.7895
Epoch: 061, Train Loss: 0.6931, Val Acc: 0.7895, Test Acc: 0.7895
Epoch: 071, Train Loss: 0.6931, Val Acc: 0.7895, Test Acc: 0.7895
Epoch: 081, Train Loss: 0.6931, Val Acc: 0.7895, Test Acc: 0.7895
Epoch: 091, Train Loss: 0.6931, Val Acc: 0.7895, Test Acc: 0.7895
Epoch: 101, Train Loss: 0.6931, Val Acc: 0.7895, Test Acc: 0.7895
Epoch: 111, Train Loss: 0.6931, Val Acc: 0.6842, Test Acc: 0.7895
Epoch: 121, Train Loss: 0.6931, Val Acc: 0.7895, Test Acc: 0.7895
Epoch: 131, Train Loss: 0.6931, Val Acc: 0.6316, Test Acc: 0.7895
Epoch: 141, Train Loss: 0.6931, Val Acc: 0.7895, Test Acc: 0.7895
Epoch: 151